In [ ]:
pip install pypdf

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install llama_index

In [ ]:
!pip install llama-index-llms-huggingface

In [19]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,DocumentSummaryIndex
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [ ]:
documents=SimpleDirectoryReader('/content/drive/MyDrive/data').load_data()
documents

In [8]:
system_prompt='''
You area chatbot and give answer when asked
'''
# a format supported by llama2
format_wrapper=SimpleInputPrompt('<|USER|>{query_str}<|ASSISTANT|>')

In [9]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-creden

In [ ]:
import torch

llm_model = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=format_wrapper,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # optional
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)


In [ ]:
%pip install llama-index-embeddings-langchain

In [12]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))


In [14]:
index=VectorStoreIndex.from_documents(documents,embed_model=embed_model)

In [15]:
index

In [16]:
query_engine=index.as_query_engine(llm=llm_model)
response1=query_engine.query("How to reshape data?")


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [17]:
print(response1)


There are several ways to reshape data in pandas, depending on the desired outcome. Here are some common methods:

1. **Gather columns into rows**: Use the `pivot()` method to turn columns into rows. For example, `df.pivot(columns='var', values='val')` will reshape a DataFrame with columns `var` and values `val` into a DataFrame with rows `var` and values `val`.
2. **Spread rows into columns**: Use the `melt()` method to turn rows into columns. For example, `df.melt(id_vars=['Length', 'Width'], var_name='Species')` will reshape a DataFrame with rows and columns into a DataFrame with columns and rows.
3. **Append rows of DataFrames**: Use the `concat()` method to append rows of DataFrames. For example, `df1 = pd.DataFrame({'Length': [10], 'Width': [2]})` and `df2 = pd.DataFrame({'Length': [5], 'Width': [3]})` will reshape two DataFrames into a single DataFrame with rows.



In [18]:
response2=query_engine.query('What is numpy?')
print(response2)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(




numpy is a library for numerical computing. It provides support for large, multi-dimensional arrays and matrices, and a wide range of high-level mathematical functions to manipulate them. It is widely used in data science, machine learning, and scientific computing.

Please let me know if you have any other questions or if there is anything else I can help you with.


In [21]:
index = DocumentSummaryIndex.from_documents(
    documents, embed_model=embed_model, llm=llm_model
)

current doc id: 5671cece-74c1-483e-96ff-1e53db0b1f35


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


current doc id: a31f9afa-4ec8-4582-86a2-0e3ffad926f9
current doc id: ee5f72f3-c434-4974-a1c2-092ff82bd80a


In [22]:
response3=query_engine.query('What are the different ways of handling missing data?')
print(response3)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



There are several ways to handle missing data in pandas, including:

1. Dropna() - This function drops rows with any column having NA/null data.
2. Fillna(value) - This function replaces all NA/null data with a specified value.
3. Dropna() and fillna() can be combined to drop rows with missing data and fill them with a specific value.
4. Select_na() - This function selects the rows with missing data and returns them as a Series.
5. Replace() - This function replaces the missing data with a specified value.
6. Impute() - This function imputes the missing data using a specified method.
7. Fill() - This function fills the missing data using a specified method.
8. Complete() - This function completes the missing data using a specified method.
9. Missing() - This function returns the missing data as a Series.
10. Na() - This function returns the missing data as a Series.

These are some of the ways to handle missing data in pandas, and the choice of method depends on the specific use case 